In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
import os
#import keras 
from keras import optimizers
from keras.applications.vgg19 import VGG19
#from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
#import numpy as np
from keras.models import Model 
from keras.layers import Input , Convolution2D , Conv2DTranspose , Reshape
from keras.layers import concatenate
#from keras import backend as K

#input_img1 = Input((224,224,3))
#input_img2 = Input((224,224,3))

model1 = VGG19(weights = "imagenet", include_top=False,input_shape=(224,224,3))
model2 = VGG19(weights = "imagenet", include_top=False,input_shape=(224,224,3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model1.layers:
   layer.trainable = False

for layer in model2.layers:
   layer.trainable = False

for layer in model2.layers:
    layer.name = layer.name + str("_2")

#Adding custom Layers 
x1 = model1.output
x2 = model2.output

# combine the output of the two branches
x = concatenate([x1, x2])

x = Convolution2D(1,(1,1), padding ="same", activation='relu')(x)

x = Conv2DTranspose(1,(1,1), strides=(16,16) , padding ="same" , activation='softmax')(x)

prediction = Reshape((112,112))(x)
# creating the final model 
VL2_model = Model(input = [model1.input , model2.input], output = prediction)

print(VL2_model.summary())

In [0]:
import cv2
import tensorflow as tf
import os
#import keras 
from keras import optimizers
from keras.applications.vgg19 import VGG19
#from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
import numpy as np
from keras.models import Model 
from keras.layers import Flatten , Dense , Dropout , concatenate
#from keras import backend as K
from tqdm import tqdm

input_train01 = '/content/drive/My Drive/Fake/Train/Input01'
input_train02 = '/content/drive/My Drive/Fake/Train/Input02'
output_train = '/content/drive/My Drive/Fake/Train/Output'

input_test01 = '/content/drive/My Drive/Fake/Test/Input01'
input_test02 = '/content/drive/My Drive/Fake/Test/Input02'
output_test = '/content/drive/My Drive/Fake/Test/Output'

##Preparamos nuestras imagenes

def data_gen(path,i):
  datagen = []
  if i==0:
    for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
      img_array = cv2.imread(os.path.join(path,img))  # convert to array #,cv2.IMREAD_GRAYSCALE
      datagen.append(img_array)  # add this to our training_data
  if i==1:
    for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
      img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array #,cv2.IMREAD_GRAYSCALE
      datagen.append(img_array)  # add this to our training_data
  return datagen

input_train01 = np.array(data_gen(input_train01,0))
input_train02 = np.array(data_gen(input_train02,0))
output_train = np.array(data_gen(output_train,1))

input_test01 = np.array(data_gen(input_test01,0))
input_test02 = np.array(data_gen(input_test02,0))
output_test = np.array(data_gen(output_test,1))

#RED NEURONAL

sess = tf.InteractiveSession()

model1 = VGG19(weights = "imagenet", include_top=False,input_shape=(224,224,3))
model2 = VGG19(weights = "imagenet", include_top=False,input_shape=(224,224,3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model1.layers:
   layer.trainable = False

for layer in model2.layers:
   layer.trainable = False

for layer in model2.layers:
    layer.name = layer.name + str("_2")

#Adding custom Layers 
x1 = model1.output
x2 = model2.output

# combine the output of the two branches
x = concatenate([x1, x2])

x = Convolution2D(1,(1,1), padding ="same", activation='relu')(x)

x = Conv2DTranspose(1,(1,1), strides=(16,16) , padding ="same" , activation='softmax')(x)

prediction = Reshape((112,112))(x)

# creating the final model 
VL2_model = Model(input = [model1.input , model2.input], output = prediction)
  
VL2_model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=0.0005),metrics=['accuracy'])
VL2_model.fit(x=[input_train01,input_train02],y=output_train,steps_per_epoch=1000,epochs=20,validation_data=[[input_test01,input_test02],output_test],validation_steps=300)

target_dir = '/content/drive/My Drive/modelo/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
    VL2_model.save('/content/drive/My Drive/modelo/modelo.h5')
    VL2_model.save_weights('/content/drive/My Drive/modelo/pesos.h5')